In [99]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

SOS_token = 0
EOS_token = 1

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('../../data/nlp/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('fr', 'en', True)
print(random.choice(pairs))

Reading lines...
Read 169657 sentence pairs
Trimmed to 12658 sentence pairs
Counting words...
Counted words:
en 4731
fr 3045
['je songe a acheter un nouveau parasol .', 'i m thinking about buying a new parasol .']


In [100]:
embedding = nn.Embedding(60, 5)
indexes = [3,6,9]
torchIndex = torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)
print(torchIndex)
print(embedding(torchIndex))

tensor([[3],
        [6],
        [9]])
tensor([[[-0.9823, -0.5065,  0.0998, -0.6540,  0.7317]],

        [[ 0.2911,  1.9907,  0.6614,  1.1899,  0.8165]],

        [[-0.6200, -1.4782, -1.1334,  0.8738, -0.5603]]],
       grad_fn=<EmbeddingBackward>)


In [101]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

teacher_forcing_ratio = 0.5

n_iters = 5 

training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]

for iter in range(1, n_iters + 1):
    training_pair = training_pairs[iter - 1]
    input_tensor = training_pair[0]
    target_tensor = training_pair[1]
    print("input tensor", input_tensor)
    print("target tensor",target_tensor)
    
print(input_tensor.size(0))

input tensor tensor([[   2],
        [2306],
        [1054],
        [ 303],
        [3098],
        [ 229],
        [2095],
        [   5],
        [   1]])
target tensor tensor([[   2],
        [   3],
        [ 819],
        [2533],
        [1175],
        [  16],
        [1849],
        [   4],
        [   1]])
input tensor tensor([[   6],
        [ 337],
        [   7],
        [ 281],
        [ 118],
        [2931],
        [   5],
        [   1]])
target tensor tensor([[   2],
        [   3],
        [ 163],
        [1730],
        [ 745],
        [   4],
        [   1]])
input tensor tensor([[153],
        [118],
        [236],
        [606],
        [  5],
        [  1]])
target tensor tensor([[ 90],
        [ 91],
        [171],
        [ 73],
        [  4],
        [  1]])
input tensor tensor([[  25],
        [  79],
        [ 730],
        [  53],
        [1509],
        [   5],
        [   1]])
target tensor tensor([[ 15],
        [ 47],
        [402],
        [561],
     

In [102]:
indexes = [1,5,7,8,4, 6]
torchIndex = torch.tensor(indexes, dtype=torch.long, device=device)
print(torchIndex)
torchIndex = torchIndex.view(1, -1)
print(torchIndex)

tensor([1, 5, 7, 8, 4, 6])
tensor([[1, 5, 7, 8, 4, 6]])


In [103]:

x = torch.tensor([3,5,6])
print(x)
y = torch.rand(4,5,1)
print(y)
print(y.shape)
y = y.unsqueeze(-1)
print(y)
print(y.shape)

tensor([3, 5, 6])
tensor([[[0.4040],
         [0.0765],
         [0.8860],
         [0.3710],
         [0.1769]],

        [[0.8206],
         [0.6776],
         [0.6237],
         [0.4009],
         [0.3440]],

        [[0.5848],
         [0.8869],
         [0.5716],
         [0.2636],
         [0.9967]],

        [[0.5107],
         [0.8953],
         [0.6219],
         [0.9482],
         [0.7885]]])
torch.Size([4, 5, 1])
tensor([[[[0.4040]],

         [[0.0765]],

         [[0.8860]],

         [[0.3710]],

         [[0.1769]]],


        [[[0.8206]],

         [[0.6776]],

         [[0.6237]],

         [[0.4009]],

         [[0.3440]]],


        [[[0.5848]],

         [[0.8869]],

         [[0.5716]],

         [[0.2636]],

         [[0.9967]]],


        [[[0.5107]],

         [[0.8953]],

         [[0.6219]],

         [[0.9482]],

         [[0.7885]]]])
torch.Size([4, 5, 1, 1])


In [104]:
import torch

torch.manual_seed(1)

x = torch.randn(2,3)

y = torch.randn(1,3)

print(x,y)

a=torch.rand((1,2))
b=torch.rand((1,2))

c=torch.stack((a,b),0)

c.size()


torch.manual_seed(1)

x = torch.randn(2,3)

print(x)

x.transpose(0,1)

x = torch.randn(2,3,4)

print(x.size())

x_p = x.permute(1,0,2) 

b = torch.Tensor(2,1)
b.shape
Out[28]: torch.Size([2, 1])

# 不加参数，去掉所有为元素个数为1的维度
b_ = b.squeeze()
b_.shape
Out[30]: torch.Size([2])

# 加上参数，去掉第一维的元素为1，不起作用，因为第一维有2个元素
b_ = b.squeeze(0)
b_.shape 
Out[32]: torch.Size([2, 1])

# 这样就可以了
b_ = b.squeeze(1)
b_.shape
Out[34]: torch.Size([2])

# 增加一个维度
b_ = b.unsqueeze(2)
b_.shape
Out[36]: torch.Size([2, 1, 1])

tensor([[ 0.6614,  0.2669,  0.0617],
        [ 0.6213, -0.4519, -0.1661]]) tensor([[-1.5228,  0.3817, -1.0276]])
tensor([[ 0.6614,  0.2669,  0.0617],
        [ 0.6213, -0.4519, -0.1661]])
torch.Size([2, 3, 4])


In [105]:
x = torch.randn(3, 2)
y = x.transpose (0, 1)
x[0, 0] = 233
print(y.shape)
print(y[0, 0])

y = y.contiguous()
print(y)

torch.Size([2, 3])
tensor(233.)
tensor([[233.0000,   0.4107,  -0.9081],
        [  2.5286,  -0.9880,   0.5423]])


In [106]:
import numpy as np
import matplotlib.pyplot as plt

TIME_STEP = 10

steps = np.linspace(0, np.pi*2, 100, dtype=np.float)

print(steps)
x1_np = np.sin(steps)
x2_np = np.cos(steps)
x3_np = np.tan(steps)

print(x1_np)






[0.         0.06346652 0.12693304 0.19039955 0.25386607 0.31733259
 0.38079911 0.44426563 0.50773215 0.57119866 0.63466518 0.6981317
 0.76159822 0.82506474 0.88853126 0.95199777 1.01546429 1.07893081
 1.14239733 1.20586385 1.26933037 1.33279688 1.3962634  1.45972992
 1.52319644 1.58666296 1.65012947 1.71359599 1.77706251 1.84052903
 1.90399555 1.96746207 2.03092858 2.0943951  2.15786162 2.22132814
 2.28479466 2.34826118 2.41172769 2.47519421 2.53866073 2.60212725
 2.66559377 2.72906028 2.7925268  2.85599332 2.91945984 2.98292636
 3.04639288 3.10985939 3.17332591 3.23679243 3.30025895 3.36372547
 3.42719199 3.4906585  3.55412502 3.61759154 3.68105806 3.74452458
 3.8079911  3.87145761 3.93492413 3.99839065 4.06185717 4.12532369
 4.1887902  4.25225672 4.31572324 4.37918976 4.44265628 4.5061228
 4.56958931 4.63305583 4.69652235 4.75998887 4.82345539 4.88692191
 4.95038842 5.01385494 5.07732146 5.14078798 5.2042545  5.26772102
 5.33118753 5.39465405 5.45812057 5.52158709 5.58505361 5.648520

In [107]:
input_seq_len = 15
output_seq_len = 15
batch_size = 1200


input_dim = 3
output_dim = 3
x = np.linspace(0, 40, 130)

def true_output_signals(x):
    x1 = 2 * np.sin(x)
    x2 = 3 * np.cos(x)
    x3 = 4 * np.tan(x)
    #x4 = 5 * np.log(x+2)

    return x1, x2, x3  #, x4

def true_input_signals(x):
    x1, x2, x3 = true_output_signals(x) #, x4
    
    y1 = 1.6*x1**4 - 2*x2 - 10 * x3
    y2 = 1.2*x2**2 * x1 + 2*x2*3 - x1*6   #+ x4 *5
    y3 = 2*x1**3 + 2*x2**3 - x1*x2 - x3   #+ 2* x4
    
    return y1, y2, y3

def noise_func(x, noise_factor = 2):
    return np.random.randn(len(x)) * noise_factor

def generate_samples_for_output(x):
    x1, x2, x3 = true_output_signals(x)   #, x4 
    return x1+noise_func(x1, 0.5), \
           x2+noise_func(x2, 0.5), \
           x3+noise_func(x3, 0.5)  #, \
           #x4+noise_func(x4, 0.5)

def generate_samples_for_input(x):
    y1, y2, y3 = true_input_signals(x)
    return y1+noise_func(y1, 2), \
           y2+noise_func(y2, 2), \
           y3+noise_func(y3, 2)

def generate_train_samples(x = train_data_x, batch_size = 10):

    total_start_points = len(x) - input_seq_len - output_seq_len
    start_x_idx = np.random.choice(range(total_start_points), batch_size)
    
    input_seq_x = [x[i:(i+input_seq_len)] for i in start_x_idx]
    output_seq_x = [x[(i+input_seq_len):(i+input_seq_len+output_seq_len)] for i in start_x_idx]
    
    input_seq_y = [generate_samples_for_input(x) for x in input_seq_x]
    output_seq_y = [generate_samples_for_output(x) for x in output_seq_x]
    
    ## return shape: (batch_size, time_steps, feature_dims)
    return np.array(input_seq_y).transpose(0, 2, 1), np.array(output_seq_y).transpose(0, 2, 1)

In [108]:
input_seq, output_seq = generate_train_samples(batch_size=batch_size)
print(input_seq.shape)
print(output_seq.shape)

(1200, 15, 3)
(1200, 15, 3)


In [109]:
import torch
import torch.nn as nn
import os
import time
from torch.utils.data import TensorDataset, DataLoader

class GRUNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers, drop_prob=0.2):
        super(GRUNet, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        self.gru = nn.GRU(input_dim, hidden_dim, n_layers, batch_first=True, dropout=drop_prob)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()
        
    def forward(self, x, h):
        out, h = self.gru(x, h)
        out = self.fc(self.relu(out[:,-1]))
        return out, h
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device)
        return hidden

In [110]:
def train(train_loader, learn_rate, hidden_dim=256, EPOCHS=150, model_type="GRU"):
    
    # Setting common hyperparameters
    #batchData = next(iter(train_loader))
    #print("batch shape" , batchData[0].shape)
    #input_dim = batchData[0].shape[2]
    #output_dim = input_dim
    n_layers = 1
    # Instantiating the models
    if model_type == "GRU":
        model = GRUNet(input_dim, hidden_dim, output_dim, n_layers)
    else:
        model = LSTMNet(input_dim, hidden_dim, output_dim, n_layers)
    model.to(device)
    
    # Defining loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learn_rate)
    
    model.train()
    print("Starting Training of {} model".format(model_type))
    epoch_times = []
    # Start training loop
    for epoch in range(1,EPOCHS+1):
        start_time = time.clock()
        h = model.init_hidden(batch_size)
        avg_loss = 0.
        counter = 0
        for x, label in train_loader:
            counter += 1
            if model_type == "GRU":
                h = h.data
            else:
                h = tuple([e.data for e in h])
            model.zero_grad()
            
            out, h = model(x.to(device).float(), h)
            loss = criterion(out, label.to(device).float())
            loss.backward()
            optimizer.step()
            avg_loss += loss.item()
            if counter%200 == 0:
                print("Epoch {}......Step: {}/{}....... Average Loss for Epoch: {}".format(epoch, counter, len(train_loader), avg_loss/counter))
        current_time = time.clock()
        print("Epoch {}/{} Done, Total Loss: {}".format(epoch, EPOCHS, avg_loss/len(train_loader)))
        print("Total Time Elapsed: {} seconds".format(str(current_time-start_time)))
        epoch_times.append(current_time-start_time)
    print("Total Training Time: {} seconds".format(str(sum(epoch_times))))
    return model

In [111]:
batch_size = 15
train_data = TensorDataset(torch.from_numpy(input_seq), torch.from_numpy(output_seq))
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size, drop_last=True)

lr = 0.0001
gru_model = train(train_loader, lr, model_type="GRU")

/usr/local/python36/lib/python3.6/site-packages/torch/nn/modules/rnn.py:51: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
/usr/local/python36/lib/python3.6/site-packages/torch/nn/modules/loss.py:431: UserWarning: Using a target size (torch.Size([15, 15, 3])) that is different to the input size (torch.Size([15, 3])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Starting Training of GRU model
Epoch 1/150 Done, Total Loss: 4813.04262008667
Total Time Elapsed: 2.5499999999999545 seconds
Epoch 2/150 Done, Total Loss: 4800.480706977844
Total Time Elapsed: 2.4300000000000637 seconds
Epoch 3/150 Done, Total Loss: 4791.738552665711
Total Time Elapsed: 2.4299999999998363 seconds
Epoch 4/150 Done, Total Loss: 4784.153258895874
Total Time Elapsed: 2.5300000000002 seconds
Epoch 5/150 Done, Total Loss: 4778.186190319061
Total Time Elapsed: 2.339999999999918 seconds
Epoch 6/150 Done, Total Loss: 4773.815562057495
Total Time Elapsed: 2.25 seconds
Epoch 7/150 Done, Total Loss: 4769.625167274475
Total Time Elapsed: 2.349999999999909 seconds
Epoch 8/150 Done, Total Loss: 4767.494065856934
Total Time Elapsed: 2.4200000000000728 seconds
Epoch 9/150 Done, Total Loss: 4765.21652765274
Total Time Elapsed: 2.5299999999999727 seconds
Epoch 10/150 Done, Total Loss: 4763.81964635849
Total Time Elapsed: 2.3299999999999272 seconds
Epoch 11/150 Done, Total Loss: 4762.5824

In [112]:
import numpy as np
path = "../../data/"
train = np.loadtxt(path +"incomeClassTrainMapped.csv", delimiter=",", skiprows=1, dtype=np.float32)
print(train.shape)

(32561, 16)


In [113]:
import torch
x_data = torch.from_numpy(train[:,0:-1])
y_data = torch.from_numpy(train[:,[-1]])
print(x_data.shape)
print(y_data.shape)

torch.Size([32561, 15])
torch.Size([32561, 1])


In [114]:
from torch.utils.data import Dataset, DataLoader, TensorDataset
deal_dataset = TensorDataset(x_data, y_data)

train_size = int(0.8 * len(deal_dataset))
test_size = len(deal_dataset) - train_size

train_dataset, test_dataset = torch.utils.data.random_split(deal_dataset, [train_size, test_size])
print("train_dataset shape", train_dataset.shape)


train_dataLoader = DataLoader(dataset = train_dataset,
                             batch_size = 32,
                             shuffle = True,
                             num_workers = 2)

for epoch in range(3):
    for index, batchData in enumerate(train_dataLoader):
        batch_x, batch_y = batchData
    print("batch_x", batch_x.shape)
    print("batch_y", batch_y.shape)
    

AttributeError: 'Subset' object has no attribute 'shape'